# Assessing the quality of similarity measures

In [1]:
import numpy as np
import networkx as nx
import xgi
import json
from tqdm import tqdm
from itertools import combinations
from math import comb
import random
from scipy.spatial import distance

from netsimile import *
from portrait_divergence import *
from hypergraphs_models import *
from hypergraphs_distances import *

# 1) Clustering by model

Here I try to compare the 4 similarity measures for hypergraphs, two higher-order (HNS, HPD) and two pairwise (NS, PD).
I generate hyprgraphs from 3 different models (ER, CM, WS) with parameters chosen randomly within ceratin intervals
and compute the distance matrix among hypergraphs according to the 4 measures. 

Then, I compare the quality of the measures as follow: I consider the models as the "real" clusters. Then I compute a modified Dunn's index, where in place of the clusters given by an eventual hierarchical clustering I consider the generative models.
So, the score for each similarity measure will be given by: minimum inter-model distance / maximum intra-model distance.

## 1.1) Generate hypergraphs

For simplicity, I restrict the hyperedge sizes to 2,3,4 and limit the choice of parameters as follow:

Number of nodes: randomly chosen between 200 and 300.

ER: average s-degree randomly chosen between 3 and 4 (same at each order of interaction).

CM: parameter 'a' regulating the power law distribution of the form: $ P(x; a) = ax^{a-1}$ (with $ 0 \le x \le 1,\ a>0 $) from which the degrees are sampled (with fixed 'k_max' and same degree distribution at every order): $k_i = k_{max} [1 - P(x; a)] + 1$. a is randomly chosen between 5 and 6.

WS: rewiring probability 'p_rew', randomly chosen between 0.2 and 0.3. The original hypergraph is a uniform ring lattice where the hyperedges are of the form (example for size 3): (0,1,2), (1,2,3), (2,3,4), and so on. 

For each model I sample n_samp realizations.

In [3]:
H_models = {}
n_samp = 100   # number of samples of each model
n_sizes = 3    # number of different edge sizes (n_sizes=3 -> s=2,3,4)
Nmin, Nmax = 200, 300

# random hypergraphs
for s in range(n_samp):
    random.seed(s)
    N = int(random.uniform(Nmin,Nmax))
    k_avg = np.round(random.uniform(3,4), 1) 
    ks = [k_avg]*n_sizes
    H_models[f'ER{s}_{N}_{k_avg}'] = stratified_er_hypergraph(N, ks, p_type='degree', seed=s)

# configuration model hypergraphs 
# with power-law degree distribution 
for s in range(n_samp):
    seed=s+n_samp
    random.seed(seed)
    N = int(random.uniform(Nmin,Nmax))
    gamma = random.uniform(2.05,2.1)
    degs = powerlaw_degree_distribution(N, gamma, seed, cutoff=30)
    deg_lists = [degs]*n_sizes
    H_models[f'CM{s}_{N}_{gamma}'] = stratified_cm_hypergraph(deg_lists, seed=seed)
    
# Watts-Strogatz hypergraphs
for s in range(n_samp):
    seed=s+n_samp*2
    random.seed(seed)
    N = int(random.uniform(Nmin,Nmax)) 
    p_rew = np.round(random.uniform(0.2,0.3), 2)
    prs = [p_rew]*n_sizes
    H_models[f'WS{s}_{N}_{p_rew}'] = stratified_ws_hypergraph(N, prs, seed=seed)

/Users/cosimoagostinelli/Work/xgi-main/xgi/generators/uniform.py:78: UserWarning: This degree sequence is not realizable. Increasing the degree of random nodes so that it is.
  warnings.warn(


In [4]:
ks_avg = [
    np.mean( list(H.degree().values()) ) for H in H_models.values()
]
ks_avg

[11.566901408450704,
 11.192488262910798,
 12.396610169491526,
 11.192825112107624,
 8.91891891891892,
 10.958015267175572,
 11.32616487455197,
 10.056034482758621,
 12.063063063063064,
 10.792682926829269,
 11.171206225680933,
 10.742857142857142,
 10.963562753036438,
 11.831111111111111,
 11.323809523809524,
 9.322033898305085,
 11.122881355932204,
 10.781746031746032,
 10.188073394495413,
 11.99625468164794,
 10.10344827586207,
 11.185185185185185,
 9.250847457627119,
 12.517123287671232,
 10.685185185185185,
 12.38396624472574,
 9.245421245421245,
 10.106060606060606,
 8.957345971563981,
 10.023622047244094,
 9.828685258964143,
 10.0,
 10.262135922330097,
 10.61089494163424,
 10.26984126984127,
 10.622047244094489,
 12.46551724137931,
 9.406716417910447,
 10.250950570342205,
 9.577272727272728,
 12.510204081632653,
 10.535864978902953,
 9.34980988593156,
 10.019704433497537,
 10.6875,
 10.0,
 11.35763888888889,
 9.83404255319149,
 9.169291338582678,
 10.135922330097088,
 10.1169354

## 1.2) Compute distances

In [5]:
labels = list(H_models.keys())
HPs, FVs = [], []
nps, fvs = [], []

for H in tqdm(H_models.values()):
    FVs.append(feature_vec(H))
    HPs.append(edge_portrait(H))
    G = xgi.to_graph(H)
    fvs.append(graph_signature(G))
    nps.append(portrait(G))
     
# compute distances between all pairs of hypergraphs
ds = distance.pdist(np.array(FVs), metric='canberra')
HNS_dists = list(ds / len(FVs[0]))
HPD_dists = [hyper_portrait_divergence(HPs[i],HPs[j]) 
             for i,j in combinations(range(len(HPs)), 2) ]

# compute distances between all pairs of networks
ds = distance.pdist(np.array(fvs), metric='canberra')
ns_dists = list(ds / len(fvs[0]))
pd_dists = [portrait_divergence(nps[i],nps[j]) 
            for i,j in combinations(range(len(nps)), 2) ]

# save results
results = (labels, HNS_dists, HPD_dists, ns_dists, pd_dists)
with open('../results/HNS_HPD_NS_PD_distances_models.json', 'w') as res_file:
    json.dump(results, res_file)

100%|█████████████████████████████████████████| 300/300 [10:14<00:00,  2.05s/it]


# 2) Clustering by hyperedge size

Now let us change perspective. Since we are interested in the higher-order nature of the systems, we consider a single hypergraph model (ER) with **fixed average projected degree** and vary the maximum hyperedge size $M$. The number of nodes $N$ can vary in the uniform interval $[200-300]$.
For each M, I sample several realizations of the model to see whether the measures can group together the hypergraphs according to their sizes of interaction.

## 2.1) Generate hypergraphs

The projected degree $\langle k_2 \rangle$ is given by the contributions of hyperedges of all sizes (as if it were a simplicial complex), according to the formula:
$$ \langle k_2 \rangle = (N-1) \sum_{s=2}^M p_s {N-2 \choose s-2}$$
where $s=2,3,...,M$ is the size of the hyperedges. Thus, to have that the projected degree is the sum of equal contributions coming from every order of interaction we have to set the relation:
$$ p_s = \frac{p_2}{N-2 \choose s-2} \ .$$
This guarantees that, for example, if we have $\langle k_2 \rangle =9$ and $M=4$ there are (on average) three 2-hyperedges, three 3-hyperedges and three 4-hyperedges).
Putting together the two equations we find the relation between $\langle k_2 \rangle$ and $p_2$:
$$ \langle k_2 \rangle = (N-1)(M-1)p_2 \ .$$

In [59]:
H_ER = {}
k2 = 20.        # average projected degree of nodes
Ms = [3,4,5,6]    # list of maximum hyperedge sizes
n_samp = 100    # number of samples for each M

for M in Ms:
    for i in tqdm(range(n_samp)):
        random.seed(i)
        N = int(random.uniform(200,300))
        ps = [ k2/( (N-1)*(M-1) ) ]
        # compute p3, p4...
        ps += [ ps[0]/comb(N-2, s-2) for s in range(3,M+1) ]
        H = stratified_er_hypergraph(N, ps, seed=i)
        H.cleanup()
        H_ER[f'ER{M}_{i}'] = H

100%|███████████████████████████████████████████| 100/100 [00:06<00:00, 14.85it/s]


In [61]:
# check that everything works as expected
for h in H_ER.values():
    g = xgi.to_graph(h)
    degrees = [val for (node, val) in g.degree()]
    print(np.mean(degrees))

s_dict = dict()
for k,h in H_ER.items():
    sizes = [len(i) for i in h.edges.members()]
    s_dict[k] = {s: sizes.count(s) for s in set(sizes)}
s_dict

19.183098591549296
19.051643192488264
19.84406779661017
19.80269058295964
19.013452914798208
19.061068702290076
19.089605734767026
19.655172413793103
19.54954954954955
20.13821138211382
20.132295719844358
19.16734693877551
18.82591093117409
20.106666666666666
18.752380952380953
19.5472972972973
20.3135593220339
19.095238095238095
18.98165137614679
20.367041198501873
18.79310344827586
19.444444444444443
19.776271186440677
20.643835616438356
18.51660516605166
19.50210970464135
18.89051094890511
18.87878787878788
18.89099526066351
19.976377952755904
18.73517786561265
20.059701492537314
19.468599033816425
18.669260700389106
19.476190476190474
18.866141732283463
20.198275862068964
19.47761194029851
19.46768060836502
18.463636363636365
19.910204081632653
20.462184873949578
19.5893536121673
18.00985221674877
19.166666666666668
19.207048458149778
20.833333333333332
18.4
19.338582677165356
18.864077669902912
20.080321285140563
19.857142857142858
19.13804713804714
19.915708812260537
20.652920962

{'ER3_0': {2: 1384, 3: 474},
 'ER3_1': {2: 1073, 3: 348},
 'ER3_2': {2: 1490, 3: 507},
 'ER3_3': {2: 1116, 3: 390},
 'ER3_4': {2: 1091, 3: 365},
 'ER3_5': {2: 1318, 3: 421},
 'ER3_6': {2: 1394, 3: 448},
 'ER3_7': {2: 1186, 3: 393},
 'ER3_8': {2: 1112, 3: 376},
 'ER3_9': {2: 1255, 3: 430},
 'ER3_10': {2: 1296, 3: 458},
 'ER3_11': {2: 1214, 3: 407},
 'ER3_12': {2: 1190, 3: 405},
 'ER3_13': {2: 1136, 3: 402},
 'ER3_14': {2: 1023, 3: 343},
 'ER3_15': {2: 1463, 3: 499},
 'ER3_16': {2: 1217, 3: 419},
 'ER3_17': {2: 1274, 3: 401},
 'ER3_18': {2: 1099, 3: 349},
 'ER3_19': {2: 1390, 3: 470},
 'ER3_20': {2: 1459, 3: 443},
 'ER3_21': {2: 1068, 3: 366},
 'ER3_22': {2: 1506, 3: 495},
 'ER3_23': {2: 1512, 3: 532},
 'ER3_24': {2: 1290, 3: 426},
 'ER3_25': {2: 1148, 3: 409},
 'ER3_26': {2: 1328, 3: 441},
 'ER3_27': {2: 1333, 3: 410},
 'ER3_28': {2: 1032, 3: 341},
 'ER3_29': {2: 1293, 3: 443},
 'ER3_30': {2: 1202, 3: 412},
 'ER3_31': {2: 1022, 3: 354},
 'ER3_32': {2: 1023, 3: 357},
 'ER3_33': {2: 1236,

## 2.2) Compute distances

In [22]:
labels = list(H_ER.keys())
HPs, FVs = [], []
nps, fvs = [], []

for H in tqdm(H_ER.values()):
    FVs.append(feature_vec(H))
    HPs.append(edge_portrait(H))
    G = xgi.to_graph(H)
    fvs.append(graph_signature(G))
    nps.append(portrait(G))

# compute distances between all pairs of hypergraphs
ds = distance.pdist(np.array(FVs), metric='canberra')
HNS_dists = list(ds / len(FVs[0]))
HPD_dists = [hyper_portrait_divergence(HPs[i],HPs[j]) 
             for i,j in combinations(range(len(HPs)), 2) ]

# compute distances between all pairs of networks
ds = distance.pdist(np.array(fvs), metric='canberra')
ns_dists = list(ds / len(fvs[0]))
pd_dists = [portrait_divergence(nps[i],nps[j]) 
            for i,j in combinations(range(len(nps)), 2) ]
    
# save results
results = (labels, HNS_dists, HPD_dists, ns_dists, pd_dists)
with open('../results/HNS_HPD_NS_PD_distances_ERs_3_4_5_6.json', 'w') as res_file:
    json.dump(results, res_file)

100%|█████████████████████████████████████████| 400/400 [12:57<00:00,  1.94s/it]
